In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model,model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten,Input,Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
import glob
from PIL import Image
from PIL import ImageOps
from keras import backend as K
from keras.callbacks import TensorBoard
import h5py
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from scipy.spatial.distance import cosine

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten,Input
color = 3;
def dog_data_load():
    print("hai")
    list = glob.glob('../dog/*')
    linumber = 1;
    
    image_li = np.empty((0,56,56,color), dtype=np.uint8)
    
    for i in list:
        #print(linumber)
        im = Image.open(i).resize((56,56))
        #print(im.shape)
        #im = im.convert("L")
        #print(im.shape)
        im_reading = np.array(im)
        
        #im_list = np.asarray(im_reading)
 
        #print("変換前")
        #print( im_reading.shape)
       
        #print("変換後")
        #print(im_reading.shape)
        
        image_li = np.append(image_li, [im_reading], axis=0)
        #print(image_li.shape)
        #print("korekore")
        #出力形式は(枚数, 1, 50, 50)
        #print(image_li.shape)
        linumber+=1
    return (image_li,image_li),(image_li,image_li)

hidden_layer1 = 256
hidden_layer2 = 64
hidden_layer3 = 3 # because you have 10 categories  
#encoding_dim = 32
input_img = Input(shape=(56,56,color))
x = Conv2D(48, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(48, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(24, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = Dense(hidden_layer1, activation='relu')(x)
x = Dense(hidden_layer2, activation='relu')(x)
encoded = Dense(hidden_layer3, activation='softmax',name="softdog")(x)

encoder = Model(input_img,encoded)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional
x = Dense(hidden_layer2, activation='relu')(encoded)
x = Dense(hidden_layer1, activation='relu')(x)
x = Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(24, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(48, (3, 3), activation='relu',padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(48, (3, 3), activation='relu',padding='same')(x)
decoded = Conv2D(3, (3, 3), activation='relu', padding='same',name="softdog2")(x)


autoencoder = Model(input_img, decoded)
AE_model = autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

(x_train, y_train),(x_test,y_test) = dog_data_load()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


x_train = np.reshape(x_train,(len(x_train),56,56,color))
x_test = np.reshape(x_test,(len(x_test),56,56,color))

history = autoencoder.fit(x_train, x_train,
                epochs=3,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='./tmp/autoencoder')]
               )

#精度をグラフに表示
loss = history.history['loss']
val_loss = history.history['val_loss']
nb_epoch = len(loss)
plt.plot(range(nb_epoch), loss, marker='.', label='loss')
plt.plot(range(nb_epoch), val_loss, marker='.', label='val_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

decoded_imgs = autoencoder.predict(x_test,batch_size=256)
print(decoded_imgs.shape)
dog_features = decoded_imgs.flatten().tolist()
print(decoded_imgs[1].shape)
# 何個表示するか
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # 変換された画像を表示
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(56, 56,color))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


autoencoder.save_weights('AE_Softmax.h5')

hai
Train on 1218 samples, validate on 1218 samples
Epoch 1/3
1218/1218 [==============================] - 16s - loss: 2.6462 - val_loss: 1.4864
Epoch 2/3
1218/1218 [==============================] - 17s - loss: 1.3593 - val_loss: 1.1731
Epoch 3/3
1024/1218 [========================>.....] - ETA: 2s - loss: 1.0772

In [ ]:
Test_image = '../test/main_53914_ccd62_detail.jpg'
Test_img = image.load_img(Test_image,target_size=(56,56))
Tx = img_to_array(Test_img)
Tx_= np.expand_dims(Tx,axis=0)
print(Tx_.shape)
Ima_score = AE_model.predict(Tx_,batch_size=32)
ax = plt.subplot(2, n, i+1+n)
plt.imshow(Ima_score.reshape(56, 56,color))
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.show()


AE_Model =AE_model.load_weights('AE_Softmax.h5')
model = Model(AE_model.input,AE_model.get_layer('softdog2').output)
features = model.predict(Tx_)
features = features.flatten().tolist()
Ima_score = AE_model.predict(Tx_,batch_size=32)
plt.imshow(Ima_score.reshape(56, 56,color))

